In [ ]:
%matplotlib inline


# Vector Quantization

Spatial VQ using K-means.

In [ ]:
!ln -sf ~/MRVC/src/logging_config.py .
!ln -sf ~/repos/quantization/deadzone_quantizer.py .
!ln -sf ~/repos/quantization/distortion.py .
!ln -sf ~/MRVC/src/image_1.py .
#!ln -sf ~/repos/quantization/information.py .

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn import cluster
import os
import pylab
import image_1 as gray_image
import distortion
import math

## Configuration

In [ ]:
home = os.environ["HOME"]
fn = home + "/MRVC/images/lena_bw/"
block_width = 16
block_height = 16
n_clusters = 256  # Number of bins
N_tries = 3  # Number of times K-means is run
N_bins = range(2, 128, 1)
gray_image.write = gray_image.debug_write # faster
#gray_image.write = gray_image.write # higher compression

## Read the image and show it

In [ ]:
img = gray_image.read(fn, 0)
gray_image.show(img, fn + "000.png")

## Example

In [ ]:
block_length = block_width*block_height
np.random.seed(seed=1)  # makes the random numbers predictable
k_means = cluster.KMeans(init="k-means++", n_clusters=n_clusters, n_init=N_tries)
blocks = []
for i in range(0, img.shape[0], block_width):
    for j in range(0, img.shape[1], block_height):
        blocks.append(np.reshape(img[i:i + block_width, j:j + block_height], block_length))
blocks = np.asarray(blocks).astype(float)
k_means.fit(blocks)
centroids = k_means.cluster_centers_.squeeze().astype(np.uint8)  # the code-book
labels = k_means.labels_  # Labels of the centroids

labels = labels.reshape(img.shape[0]//block_height, img.shape[1]//block_width)
img_dequantized = np.empty_like(img)
for i in range(0, img.shape[0], block_width):
    for j in range(0, img.shape[1], block_height):
        img_dequantized[i:i + block_width, j:j + block_height] = centroids[labels[i//block_width,j//block_height]].reshape(block_height, block_width)

In [ ]:
gray_image.show(img_dequantized, "Dequantized Image")
assert len(centroids) == n_clusters
print("centroids =\n", centroids)
bits_per_block = int(math.log(n_clusters)/math.log(2))
print(f"{len(centroids)} centroids ({bits_per_block} bits/block)")
blocks_in_y = img.shape[0]//block_height
blocks_in_x = img.shape[1]//block_width
print(f"{blocks_in_y}x{blocks_in_x} blocks (vectors) in the image")
number_of_blocks = blocks_in_y*blocks_in_x
total_number_of_bits = number_of_blocks*bits_per_block
print(f"total number of output bytes = {total_number_of_bits//8}")

In [ ]:
def RD_curve(img, N_bins):
    points = []
    flatten_img = img.reshape((-1, 1))  # flatten
    for n in N_bins:
        k_means = cluster.KMeans(n_clusters=n, n_init=N_tries)
        k_means.fit(flatten_img)
        centroids = k_means.cluster_centers_.squeeze().astype(np.uint8)  # Centroids
        k = k_means.labels_.astype(np.uint8)  # Labels of the centroids
        #y = np.choose(k, centroids)
        y = centroids[k[range(len(k))]]
        y.shape = img.shape
        k.shape = img.shape
        print("Quantization indexes: ", np.unique(k))
        rate = gray_image.write(k, "/tmp/" + str(n) + '_', 0)*8/k.size
        _distortion = distortion.RMSE(img, y)
        if n<16:
            plt.imshow(y, cmap=plt.cm.gray, vmin=0, vmax=256)
            plt.show()
        points.append((rate, _distortion))
        print(f"n={n:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

RD_points = RD_curve(img, N_bins)

In [ ]:
pylab.figure(dpi=150)
pylab.scatter(*zip(*RD_points), label=f"K-means PNG", s=1, marker='.')
#pylab.plot(*zip(*RD_points), c='m', marker='x', label=f"K-means", linestyle="dotted")
pylab.title(f"Rate/Distortion Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open(f"Lloyd-Max_RD_points.txt", 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")